# Synthetic Actions

Generate a dataset of questions or actions a user may wish to perform with their smart home.

In [1]:
import openai

from home_assistant_datasets.secrets import get_secret
from home_assistant_datasets import model_client

MODEL_ID = "gpt-3.5-turbo-0125"

openai = openai.OpenAI(api_key=get_secret("openai_key"))
model = model_client.ModelClient(openai, MODEL_ID)

# Synthetic Action Generation

In [2]:
import pathlib
import yaml

DATASET_DIR = pathlib.Path("../datasets/")
SEEDS_DIR = DATASET_DIR / "seeds"
HOME_AREAS_FILE = SEEDS_DIR / "actions.yaml"

with open(HOME_AREAS_FILE) as f:
    seed_actions = yaml.load(f.read(), Loader=yaml.Loader)

In [12]:
ACTIONS_PROMPT = f"""
You are an expert in smart home automation and are generating data used to
evaluate the performance of a smart home system Home Assistant on tasks like
summarization, performing actions, or being an independent agent managing
automations and maintenace tasks.

A device in Home Assistant represents a physical or virtual object. Users interact
with a device through entities. For example a smart light bulb may have an entity
that represents the light bulb. A smart thermostat may have a climate entity and
sensor and humidity sensors. A smart dishwasher device may be represented as a few
sensors, a select entity for changing the mode, and switch entities.

You are going to help generate the evaluation dataset for home data that
consists of details about the home and its location, areas of the home, and
devices and entities present. You will mostly generate questions that are
relevant for evaluation against the smart home, but also occasioanlly you should
throw in some actions that are not relevant so we can test those too.

The following are example actions that could be generated for a smart home:
{seed_actions}
"""

In [13]:
response = model.complete(ACTIONS_PROMPT, "Generate 20 possible actions")

In [14]:
print(response)

1. Summarize the energy consumption of all devices in the home.
2. Evaluate the indoor air quality and suggest actions to improve it.
3. Turn off the irrigation system if it is raining outside.
4. Notify the user if a window or door is left open while the HVAC system is running.
5. Determine the optimal temperature settings for energy efficiency based on weather forecast.
6. Check if all the devices in the home are connected to the WiFi network.
7. Turn on the porch lights at sunset and turn them off at sunrise.
8. Notify the user if the refrigerator door has been left open for an extended period.
9. Evaluate the security camera footage and notify the user of any suspicious activity.
10. Adjust the thermostat settings based on the number of occupants in the home.
11. Activate the robot vacuum to clean specific rooms at scheduled times.
12. Notify the user if the garage door is open and it's starting to rain.
13. Determine if any devices in the home need software updates or maintenance.

In [19]:
import itertools
import random
from tqdm.auto import tqdm

N_DATAPOINTS = 10
DEVICE_ENTITIES_YAML = DATASET_DIR / "device-entities.yaml"
ACTIONS_YAML = DATASET_DIR / "actions.yaml"

with open(DEVICE_ENTITIES_YAML, "r") as f:
    content = f.read()
    homes = list(yaml.load_all(content, Loader=yaml.Loader))

random.shuffle(homes)

if len(homes) > N_DATAPOINTS:
    homes = homes[:N_DATAPOINTS]

skipped = 0

with open(ACTIONS_YAML, "w") as area_output:
    with tqdm(total=len(homes)) as pbar:
        for home in homes:
            home_yaml = yaml.dump(home, sort_keys=False)
            response = model.complete(ACTIONS_PROMPT, "Generate 20 possible actions for this home\n{home_yaml}\n\nactions:")

            try:
                response_obj = yaml.safe_load(response)
            except yaml.YAMLError:
                skipped += 1
                continue

            updated_home = home.copy()
            updated_home.update({"actions": response_obj})
            area_output.write(yaml.dump(updated_home, explicit_start=True, sort_keys=False))
            pbar.set_description(f"Skipped {skipped}")
            pbar.update(1)


  0%|          | 0/5 [00:00<?, ?it/s]

Skipped 0: 100%|██████████| 5/5 [00:17<00:00,  3.49s/it]
